In [30]:
import torch
import numpy as np
import soundfile as sf
import re
from IPython.display import Audio as IPyAudio
from transformers import SpeechT5ForTextToSpeech, SpeechT5Processor, SpeechT5HifiGan
from speechbrain.inference.classifiers import EncoderClassifier

DEVICE = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Device: {DEVICE}")

Device: cuda


In [31]:
model_inf = SpeechT5ForTextToSpeech.from_pretrained("./speecht5_tts_colombian_final").to(DEVICE)
processor_inf = SpeechT5Processor.from_pretrained("./speecht5_tts_colombian_final")
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan").to(DEVICE)
model_inf.eval()

spk_model = EncoderClassifier.from_hparams(
    source="speechbrain/spkrec-xvect-voxceleb",
    run_opts={"device": DEVICE}
)

REF_AUDIO_PATH = "./speaker_reference.wav"

INFO:speechbrain.utils.fetching:Fetch hyperparams.yaml: Fetching from HuggingFace Hub 'speechbrain/spkrec-xvect-voxceleb' if not cached
INFO:speechbrain.utils.fetching:Fetch custom.py: Fetching from HuggingFace Hub 'speechbrain/spkrec-xvect-voxceleb' if not cached
INFO:speechbrain.utils.fetching:Fetch embedding_model.ckpt: Fetching from HuggingFace Hub 'speechbrain/spkrec-xvect-voxceleb' if not cached
INFO:speechbrain.utils.fetching:Fetch mean_var_norm_emb.ckpt: Fetching from HuggingFace Hub 'speechbrain/spkrec-xvect-voxceleb' if not cached
INFO:speechbrain.utils.fetching:Fetch classifier.ckpt: Fetching from HuggingFace Hub 'speechbrain/spkrec-xvect-voxceleb' if not cached
INFO:speechbrain.utils.fetching:Fetch label_encoder.txt: Fetching from HuggingFace Hub 'speechbrain/spkrec-xvect-voxceleb' if not cached
INFO:speechbrain.utils.parameter_transfer:Loading pretrained files for: embedding_model, mean_var_norm_emb, classifier, label_encoder
/usr/local/lib/python3.11/dist-packages/speechb

In [32]:
from datasets import load_dataset, concatenate_datasets, Audio

dsA = load_dataset("ZoniaChatbot/google-colombian-spanish-female-16000-1523")["train"]
dsB = load_dataset("ylacombe/google-colombian-spanish", "female")["train"]

cols = list(set(dsA.column_names) & set(dsB.column_names))
ds = concatenate_datasets([dsA.select_columns(cols), dsB.select_columns(cols)])
ds = ds.cast_column("audio", Audio(sampling_rate=16000))

embeddings = []
for i in range(20):
    audio = np.array(ds[i]["audio"]["array"], dtype=np.float32)
    waveform = torch.tensor(audio).unsqueeze(0).float().to(DEVICE)
    with torch.no_grad():
        emb = spk_model.encode_batch(waveform)
    embeddings.append(emb.squeeze().cpu().numpy())

spk_emb_avg = np.mean(embeddings, axis=0)
spk_emb_avg = spk_emb_avg / np.linalg.norm(spk_emb_avg)

np.save("./speaker_embedding.npy", spk_emb_avg)
print("Speaker embedding guardado en ./speaker_embedding.npy")

Speaker embedding guardado en ./speaker_embedding.npy


In [33]:
spk_emb_avg = torch.tensor(
    np.load("./speaker_embedding.npy"), dtype=torch.float32
).unsqueeze(0).to(DEVICE)

In [35]:
import torch, numpy as np, soundfile as sf, re
from IPython.display import Audio as IPyAudio
from transformers import SpeechT5ForTextToSpeech, SpeechT5Processor, SpeechT5HifiGan

DEVICE = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"

model_inf = SpeechT5ForTextToSpeech.from_pretrained("./speecht5_tts_colombian_final").to(DEVICE)
processor_inf = SpeechT5Processor.from_pretrained("./speecht5_tts_colombian_final")
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan").to(DEVICE)
model_inf.eval()

spk_emb_avg = torch.tensor(
    np.load("./speaker_embedding.npy"), dtype=torch.float32
).unsqueeze(0).to(DEVICE)